#### Overview of the problem
using %timeit we are basically trying to find the various run-times that matmul takes in python and cython
by suitable transformations. We obtain inferences from the results by plotting gflops and run-times

In [ ]:
#Step 1
import numpy as np
# • Construct two numpy matrices of size 10x10 each - the entries should be random numbers.

u = np.random.random((10,10))
v = np.random.random((10,10))
def matrix_multiply(u, v):
    m, n = u.shape
    n, p = v.shape
    res = np.zeros((m, p))
    for i in range(m):
        for j in range(p):
            res[i,j] = 0
            for k in range(n):
                res[i,j] += u[i,k] * v[k,j]
    return res

# Use the %timeit special command to find the time required for multiplying the matrices.
%timeit matrix_multiply(u, v)
# Multiply the same matrices using the u @ v or np.matmul(u, v) notation and measure the time required.
%timeit u@v

#### Estimate the total number of multiplications required for this matrix computation: 
1000
#### total number of floating point operations: 
2000
#### estimate the GFLOPS: 
0.0037 GFLOPS

### Step2
#### Use the lscpu command to find the maximum operating frequency of the CPU on your system:
CPU MHz: 2099.998
#### Estimate the maximum FLOPS that may be achievable using a single processor core.
2.099 GFLOPS
#### Comment on how well this compares against the numbers you found in Step 1:
On comparison it is clear that the CPU is capable of giving 567 times the number of GFLOPS we got in step1

In [ ]:
#Step3-part1
# Repeat step 1, but doubling the matrix sizes for each run.
import numpy as np
import matplotlib.pyplot as plt

def matrix_multiply(u, v):
    m, n = u.shape
    n, p = v.shape
    res = np.zeros((m, p))
    for i in range(m):
        for j in range(p):
            res[i,j] = 0
            for k in range(n):
                res[i,j] += u[i,k] * v[k,j]
    return res

matrix_size=[]
a=[]
b=[]
for i in range(0,8):
    m_s= 2**i
    u = np.random.random((2**i,2**i))
    v = np.random.random((2**i,2**i))
    matrix_size.append(m_s)
    a1= %timeit -o -n 1 -r 1 matrix_multiply(u, v)
    a.append(a1.best)
    b1= %timeit -o -n 1 -r 1 np.matmul(u, v)
    b.append(b1.best)

# Plot the measured times for the above code as well as the numpy matmul and comment on the results:
plt.plot(matrix_size,a,matrix_size,b)
plt.xlabel("matrix size")
plt.ylabel("run time")
plt.show()

# do they follow an expected path?
'''no'''

In [ ]:
#Step3-part2
# You should be able to run np.matmul for much higher matrix sizes than the above code.
import numpy as np
import matplotlib.pyplot as plt

for i in range(0,8):
    u = np.random.random((100*(2**i),100*(2**i)))
    v = np.random.random((100*(2**i),100*(2**i)))
    %timeit -o -n 1 -r 1 np.matmul(u, v)

In [ ]:
#Step3-part3
# Plot the estimated GFLOPS from both approaches to matrix multiplication and compare against the theoretical estimates
import numpy as np
import matplotlib.pyplot as plt

def matrix_multiply(u, v):
    m, n = u.shape
    n, p = v.shape
    res = np.zeros((m, p))
    for i in range(m):
        for j in range(p):
            res[i,j] = 0
            for k in range(n):
                res[i,j] += u[i,k] * v[k,j]
    return res

matrix_size=[]
a=[]
b=[]
for i in range(0,8):
    m_s= 2**i
    u = np.random.random((2**i,2**i))
    v = np.random.random((2**i,2**i))
    matrix_size.append(m_s)
    a1= %timeit -o -n 1 -r 1 matrix_multiply(u, v)
    a1= ((2**i)**3)/(a1.best*1e9)
    a.append(a1)
    b1= %timeit -o -n 1 -r 1 np.matmul(u, v)
    b1= ((2**i)**3)/(b1.best*1e9)
    b.append(b1)

plt.plot(matrix_size,a,matrix_size,b)
plt.show()

### How high are you able to go with this ?(step3-partb)
on repeated trial and errors, considering the fact that 40s is a very high run time, a 12,800x12,800 matrix is the highest one upon reaching the 8th iteration of doubling the matrix


In [ ]:
%load_ext Cython

In [ ]:
%%cython --annotate
#Step4
#Convert the matrix_multiply function to cython. 

import cython
import numpy as np

def cy_matmul(u, v, res):
    cdef int m, n, p
    cdef int i, j, k
    m = u.shape[0]
    n = u.shape[1]
    p = v.shape[1]
    # res = np.zeros((m, p))
    for i in range(m):
        for j in range(p):
            res[i,j] = 0
            for k in range(n):
                res[i,j] += u[i,k] * v[k,j]
    return res

In [ ]:
#time it for 50x50 matrices. Compare with the original.
u = np.float32(np.random.random((50,50)))
v = np.float32(np.random.random((50,50)))
res = np.zeros((50, 50), dtype=np.float32)
cython_mm= %timeit -o -n 1 -r 1 cy_matmul(u, v, res)
print("\n")
print(f"time for 50x50 matrices for cython in seconds: {cython_mm.best}\n")


In [ ]:
%load_ext Cython

In [ ]:
%%cython --annotate
#Step5
# On applying each of the following transformations as asked found out that declaring matrices as double[:,:] gave the best optimal time
import cython
import numpy as np

# @cython.boundscheck(False)
# @cython.wraparound(False)
def cy_matmul(double[:,:] u, double[:,:] v, double[:,:] res):

# def cy_matmul(u, v, res):
    # cdef int m, n, p
    # cdef int i, j, k
    m = u.shape[0]
    n = u.shape[1]
    p = v.shape[1]
    res = np.zeros((m, p))
    for i in range(m):
        for j in range(p):
            res[i,j] = 0
            for k in range(n):
                res[i,j] += u[i,k] * v[k,j]
    return res

In [ ]:
u = np.double(np.random.random((50,50)))
v = np.double(np.random.random((50,50)))
res = np.zeros((50, 50), dtype=np.double())
cython_mm= %timeit -o -n 1 -r 1 cy_matmul(u, v, res)
python_mm= %timeit -o -n 1 -r 1 u@v
print("\n")
# On applying each of the following transformations as asked found out that declaring matrices as double[:,:] gave the best optimal time
print(f"time for 50x50 matrices for cython in seconds: {cython_mm.best}\n")

def matrix_multiply(u, v):
    m, n = u.shape
    n, p = v.shape
    res = np.zeros((m, p))
    for i in range(m):
        for j in range(p):
            res[i,j] = 0
            for k in range(n):
                res[i,j] += u[i,k] * v[k,j]
    return res

m_m= %timeit -o -q -n 1 -r 1 matrix_multiply(u, v)

print(f"time for 50x50 matrices for python in seconds: {m_m.best}\n")

### Step5
#### transformation1:
#### • Declare each of the variables i, j, k, m, n, p as int types
optimal time: 140ms

#### transformation2:
#### • Use the decorator function @cython.boundscheck(False). 
optimal time: 52.7ms
#### What does this do?
It is used to disable array bounds checking for a specific function or method. 
By disabling bounds checking, you are telling Cython to assume that the indices 
you use to access arrays or sequences are always within valid bounds. 

#### transformation3:
#### • Declare the input variables to be of type double[:, :]
optimal time: 0.173ms

On applying each of the following transformations as asked found out that declaring matrices as double[:,:] gave the best optimal time

#### transformation4:
#### • Declare res also to be an argument of the function, of the same double[:,:] type, and make sure that res is initialized to a zero array before calling.
optimal time: 0.416ms

#### transformation5:
#### • Change the data type to float[:,:] and repeat the experiments. Does this change anything?
optimal time: 0.519ms
on finding the type(u[0][0]) we find that it outputs numpy.float64 which clearly tells float is the default datatype.